# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

Tutorials to use OpenAI DRL to trade multiple stocks in one Jupyter Notebook | Presented at NeurIPS 2020: Deep RL Workshop

* This blog is based on our paper: FinRL: A Deep Reinforcement Learning Library for Automated Stock Trading in Quantitative Finance, presented at NeurIPS 2020: Deep RL Workshop.
* Check out medium blog for detailed explanations: https://towardsdatascience.com/finrl-for-quantitative-finance-tutorial-for-multiple-stock-trading-7b00763b7530
* Please report any issues to our Github: https://github.com/AI4Finance-LLC/FinRL-Library/issues
* **Pytorch Version** 



# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5.Build Environment](#4)  
    * [5.1. Training & Trade Data Split](#4.1)
    * [5.2. User-defined Environment](#4.2)   
    * [5.3. Initialize Environment](#4.3)    
* [6.Implement DRL Algorithms](#5)  
* [7.Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
    * [7.3. Baseline Stats](#6.3)   
    * [7.3. Compare to Stock Market Index](#6.4)             

<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A includes three actions: a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying one stock. Also, an action can be carried upon multiple shares. We use
an action space {−k, ..., −1, 0, 1, ..., k}, where k denotes the number of shares. For example, "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- ASSUMES USING DOCKER, see readme for instructions

<a id='1.1'></a>
## 2.1. Add FinRL to your path. You can of course install it as a pipy package, but this is for development purposes.


In [1]:
# import sys

# sys.path.append("..")

In [2]:
# import pandas as pd
# print(pd.__version__)


<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them. 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline
import datetime

from finrl.apps import config
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
# from finrl.finrl_meta.env_stock_trading.env_stocktrading_cashpenalty import StockTradingEnvCashpenalty\
from finrl.finrl_meta.env_stock_trading.env_stocktrading_cashpenalty2 import StockTradingEnvCashpenalty
from finrl.drl_agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts


# import sys
# sys.path.append("../FinRL-Library")

Logging to C:\Users\left5\AppData\Local\Temp\SB3-2021-12-15-14-48-24-598645


d:\code\python\pyfolio\pyfolio\pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [4]:
from lutils.stock import LTdxHq

<a id='1.4'></a>
## 2.4. Create Folders

In [5]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).




-----
class YahooDownloader:
    Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API


In [6]:
# from config.py start_date is a string
config.START_DATE

'2009-01-01'

In [7]:
# from config.py end_date is a string
config.END_DATE

'2021-10-31'

In [8]:
# print(config.DOW_30_TICKER)
stock_codes = ['600603', '600583', '600582', '600565', '600558', '600551',
       '600509', '600503', '600488', '600469', '600439', '600433',
       '600428', '600425', '600387', '600383', '600382', '600369',
       '600339', '600337', '600335', '600326', '600320', '600312',]

#        '002666', '002654', '002641', '002628', '002627', '002620',
#        '002608', '002593', '002586', '002582', '002566', '002560',]

In [9]:
# df = YahooDownloader(start_date = '2009-01-01',
#                      end_date = '2021-12-13',
#                      ticker_list = config.DOW_30_TICKER).fetch_data()

In [10]:
ltdxhq = LTdxHq()

indexs = None
dfs = []
for code in stock_codes:
    df = ltdxhq.get_k_data_daily(code, start='2010-01-01')
    
    if indexs is None:
        indexs = df.index
    else:
        indexs = indexs.union(df.index)
    
#     df = df.assign(date = df.index)
#     df = df.assign(day = df.index.weekday)
#     df.date = df.date.dt.strftime('%Y-%m-%d')
    df = df.assign(tic = code)
#     df.index = range(df.shape[0])
    
    dfs.append(df)
    print('----------- over %s %s -----------' % (code, df.index.min()))

for i, df in enumerate(dfs):
    df = df.reindex(indexs)
    df = df.assign(date = df.index)
    df = df.assign(day = df.index.weekday)
    df.index = range(df.shape[0])
    
    dfs[i] = df.ffill()
    
df = pd.concat(dfs)
# df.index = range(df.shape[0])

ltdxhq.close()

----------- over 600603 2010-01-04 00:00:00 -----------
----------- over 600583 2010-01-04 00:00:00 -----------
----------- over 600582 2010-01-04 00:00:00 -----------
----------- over 600565 2010-01-04 00:00:00 -----------
----------- over 600558 2010-01-04 00:00:00 -----------
----------- over 600551 2010-01-04 00:00:00 -----------
----------- over 600509 2010-01-04 00:00:00 -----------
----------- over 600503 2010-01-04 00:00:00 -----------
----------- over 600488 2010-01-04 00:00:00 -----------
----------- over 600469 2010-01-04 00:00:00 -----------
----------- over 600439 2010-01-04 00:00:00 -----------
----------- over 600433 2010-01-04 00:00:00 -----------
----------- over 600428 2010-01-04 00:00:00 -----------
----------- over 600425 2010-01-04 00:00:00 -----------
----------- over 600387 2010-01-04 00:00:00 -----------
----------- over 600383 2010-01-04 00:00:00 -----------
----------- over 600382 2010-01-04 00:00:00 -----------
----------- over 600369 2010-01-04 00:00:00 ----

In [11]:
# df1 = dfs[0].reindex(indexs)
# df2 = dfs[1].reindex(indexs)

In [12]:
# df1.ffill()['2010-01-02':]

In [13]:
# df3 = df2.ffill()

In [14]:
# dfs[1][dfs[-3].close.isna()]

In [15]:
# def clean_data(data):
#     df = data.copy()
#     df = df.sort_values(["date", "tic"], ignore_index=True)
#     df.index = df.date.factorize()[0]
#     merged_closes = df.pivot_table(index="date", columns="tic", values="close")
# #     print(merged_closes)
# #     merged_closes = merged_closes.dropna(axis=1)
# #     print(merged_closes)
#     tics = merged_closes.columns
# #     print(tics)
#     df = df[df.tic.isin(tics)]
# #     print(df)
#     return df

In [16]:
# print(clean_data(df))

# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [17]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
                    use_turbulence=False,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators


In [18]:
processed['log_volume'] = np.log(processed.volume*processed.close)
processed['change'] = (processed.close-processed.open)/processed.close
processed['daily_variance'] = (processed.high-processed.low)/processed.close
processed.head()

,open,close,high,low,volume,amount,tic,date,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,log_volume,change,daily_variance
0,11.38,11.41,11.47,11.22,13228405.0,205262496.0,600312,2010-01-04,0,0.0,11.926838,11.163162,100.0,66.666667,100.0,11.41,11.41,18.832367,0.002629,0.021911
2904,8.12,7.99,8.17,7.98,19659656.0,201463792.0,600320,2010-01-04,0,0.0,11.926838,11.163162,100.0,66.666667,100.0,7.99,7.99,18.872270,-0.016270,0.023780
5808,8.81,9.01,9.18,8.77,16012911.0,239799376.0,600326,2010-01-04,0,0.0,11.926838,11.163162,100.0,66.666667,100.0,9.01,9.01,18.787241,0.022198,0.045505
8712,4.42,4.48,4.63,4.42,11014190.0,83350864.0,600335,2010-01-04,0,0.0,11.926838,11.163162,100.0,66.666667,100.0,4.48,4.48,17.714318,0.013393,0.046875
11616,3.46,3.42,3.52,3.37,12525719.0,117352024.0,600337,2010-01-04,0,0.0,11.926838,11.163162,100.0,66.666667,100.0,3.42,3.42,17.572935,-0.011696,0.043860


<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

## Training data split: 2009-01-01 to 2016-01-01
## Trade data split: 2016-01-01 to 2021-01-01

DRL model needs to update periodically in order to take full advantage of the data, ideally we need to retrain our model yearly, quarterly, or monthly. We also need to tune the parameters along the way, in this notebook I only use the in-sample data from 2009-01 to 2016-01 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [19]:
train = data_split(processed, '2009-01-01','2021-01-01') # '2009-01-01','2016-01-01'
trade = data_split(processed, '2021-01-01','2021-12-13') # '2016-01-01','2021-01-01'
print(len(train))
print(len(trade))

64176
5472


In [20]:
print(StockTradingEnvCashpenalty.__doc__)


    A stock trading environment for OpenAI gym
    This environment penalizes the model for not maintaining a reserve of cash.
    This enables the model to manage cash reserves in addition to performing trading procedures.
    Reward at any step is given as follows
        r_i = (sum(cash, asset_value) - initial_cash - max(0, sum(cash, asset_value)*cash_penalty_proportion-cash))/(days_elapsed)
        This reward function takes into account a liquidity requirement, as well as long-term accrued rewards.
    Parameters:
        df (pandas.DataFrame): Dataframe containing data
        buy_cost_pct (float): cost for buying shares
        sell_cost_pct (float): cost for selling shares
        hmax (int, array): maximum cash to be traded in each trade per asset. If an array is provided, then each index correspond to each asset
        discrete_actions (bool): option to choose whether perform dicretization on actions space or not
        shares_increment (int): multiples number of shares ca

#### state space
The state space of the observation is as follows 

`start_cash, <owned_shares_of_n_assets>, <<indicator_i_for_asset_j> for j in assets>`

indicators are any daily measurement you can achieve. Common ones are 'volume', 'open' 'close' 'high', 'low'.
However, you can add these as needed, 
The feature engineer adds indicators, and you can add your own as well. 


In [21]:
processed[processed['cci_30'] == -np.inf]

,open,close,high,low,volume,amount,tic,date,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,log_volume,change,daily_variance
68202,4.64,5.08,5.32,4.63,24560764.0,285484032.0,600603,2015-10-29,3,-0.049389,5.08,5.08,44.297708,-inf,31.019858,5.08,5.295167,18.641972,0.086614,0.135827
68205,4.64,5.08,5.32,4.63,24560764.0,285484032.0,600603,2015-11-03,1,-0.039794,5.08,5.08,44.297708,-inf,31.019858,5.08,5.297833,18.641972,0.086614,0.135827
68208,4.64,5.08,5.32,4.63,24560764.0,285484032.0,600603,2015-11-06,4,-0.031946,5.08,5.08,44.297708,-inf,31.019858,5.08,5.323333,18.641972,0.086614,0.135827
68211,4.64,5.08,5.32,4.63,24560764.0,285484032.0,600603,2015-11-11,2,-0.025575,5.08,5.08,44.297708,-inf,31.019858,5.08,5.316500,18.641972,0.086614,0.135827
68214,4.64,5.08,5.32,4.63,24560764.0,285484032.0,600603,2015-11-16,0,-0.020433,5.08,5.08,44.297708,-inf,31.019858,5.08,5.276833,18.641972,0.086614,0.135827
68217,4.64,5.08,5.32,4.63,24560764.0,285484032.0,600603,2015-11-19,3,-0.016300,5.08,5.08,44.297708,-inf,31.019858,5.08,5.226000,18.641972,0.086614,0.135827
68220,4.64,5.08,5.32,4.63,24560764.0,285484032.0,600603,2015-11-24,1,-0.012987,5.08,5.08,44.297708,-inf,31.019858,5.08,5.187833,18.641972,0.086614,0.135827
68223,4.64,5.08,5.32,4.63,24560764.0,285484032.0,600603,2015-11-27,4,-0.010339,5.08,5.08,44.297708,-inf,31.019858,5.08,5.155833,18.641972,0.086614,0.135827
68228,4.64,5.08,5.32,4.63,24560764.0,285484032.0,600603,2015-12-04,4,-0.007059,5.08,5.08,44.297708,-inf,31.019858,5.08,5.102500,18.641972,0.086614,0.135827
68231,4.64,5.08,5.32,4.63,24560764.0,285484032.0,600603,2015-12-09,2,-0.005612,5.08,5.08,44.297708,-inf,31.019858,5.08,5.080000,18.641972,0.086614,0.135827


In [22]:
processed.head()

,open,close,high,low,volume,amount,tic,date,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,log_volume,change,daily_variance
0,11.38,11.41,11.47,11.22,13228405.0,205262496.0,600312,2010-01-04,0,0.0,11.926838,11.163162,100.0,66.666667,100.0,11.41,11.41,18.832367,0.002629,0.021911
2904,8.12,7.99,8.17,7.98,19659656.0,201463792.0,600320,2010-01-04,0,0.0,11.926838,11.163162,100.0,66.666667,100.0,7.99,7.99,18.872270,-0.016270,0.023780
5808,8.81,9.01,9.18,8.77,16012911.0,239799376.0,600326,2010-01-04,0,0.0,11.926838,11.163162,100.0,66.666667,100.0,9.01,9.01,18.787241,0.022198,0.045505
8712,4.42,4.48,4.63,4.42,11014190.0,83350864.0,600335,2010-01-04,0,0.0,11.926838,11.163162,100.0,66.666667,100.0,4.48,4.48,17.714318,0.013393,0.046875
11616,3.46,3.42,3.52,3.37,12525719.0,117352024.0,600337,2010-01-04,0,0.0,11.926838,11.163162,100.0,66.666667,100.0,3.42,3.42,17.572935,-0.011696,0.043860


In [23]:
information_cols = ['daily_variance', 'change', 'log_volume', 'close','day', 
                    'macd', 'rsi_30', 'boll_ub', 'dx_30'] # cci_30

e_train_gym = StockTradingEnvCashpenalty(
    df = train,
    initial_amount = 50e4, # 1e6, 
    hmax = 2000, 
    turbulence_threshold = None, 
    currency='￥',
    buy_cost_pct=3e-3,
    sell_cost_pct=3e-3,
    cash_penalty_proportion=0.2,
    cache_indicator_data=True,
    daily_information_cols = information_cols,
    print_verbosity = 500, 
    random_start = True,
    ) # patient = True

e_trade_gym = StockTradingEnvCashpenalty(
    df = trade,
    initial_amount = 50e4, # 1e6, 
    hmax = 2000, 
    turbulence_threshold = None, 
    currency='￥',
    buy_cost_pct=3e-3,
    sell_cost_pct=3e-3,
    cash_penalty_proportion=0.2,
    cache_indicator_data=True,
    daily_information_cols = information_cols, 
    print_verbosity = 500, 
    random_start = False,
    ) # patient = True

caching data
data cached!
caching data
data cached!


## Environment for Training
There are two available environments. The multiprocessing and the single processing env. 
Some models won't work with multiprocessing. 

```python
# single processing
env_train, _ = e_train_gym.get_sb_env()


#multiprocessing
env_train, _ = e_train_gym.get_multiproc_env(n = <n_cores>)
```


In [24]:
# for this example, let's do multiprocessing with n_cores-2
import multiprocessing

# n_cores = multiprocessing.cpu_count() - 2
# n_cores = 24
# print(f"using {n_cores} cores")

#this is our training env. It allows multiprocessing
# env_train, _ = e_train_gym.get_multiproc_env(n = n_cores)
env_train, _ = e_train_gym.get_sb_env()

#this is our observation environment. It allows full diagnostics
env_trade, _ = e_trade_gym.get_sb_env()

<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [25]:
agent = DRLAgent(env = env_train)

### Model PPO


In [26]:
from torch.nn import Softsign, ReLU

ppo_params ={'n_steps': 256, 
             'ent_coef': 0.0, 
             'learning_rate': 0.0000005, 
             'batch_size': 1024, 
             'gamma': 0.99}

policy_kwargs = {
#     "activation_fn": ReLU, 
    "net_arch": [1024 for _ in range(10)], 
#     "squash_output": True
}

model = agent.get_model('ppo', # ppo, a2c
                        model_kwargs = ppo_params, 
                        policy_kwargs = policy_kwargs, verbose = 0)

# model = model.load("scaling_reward.model", env = env_train)

{'n_steps': 256, 'ent_coef': 0.0, 'learning_rate': 5e-07, 'batch_size': 1024, 'gamma': 0.99}


d:\soft\python\python37\lib\site-packages\stable_baselines3\ppo\ppo.py:138: UserWarning: You have specified a mini-batch size of 1024, but because the `RolloutBuffer` is of size `n_steps * n_envs = 256`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 256
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=256 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"


In [27]:
model.learn(total_timesteps = 5000000, 
            eval_env = env_trade, 
            eval_freq = 500, 
            log_interval = 1, 
            tb_log_name = 'env_cashpenalty_highlr', 
            n_eval_episodes = 1)

EPISODE|STEPS|TERMINAL_REASON|CASH           |TOT_ASSETS     |TERMINAL_REWARD_unsc|GAINLOSS_PCT|CASH_PROPORTION
   1| 202|CASH SHORTAGE  |￥8,009         |￥507,219       |-0.08536% |1.44378%  |1.58%     


d:\soft\python\python37\lib\site-packages\stable_baselines3\common\evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


EPISODE|STEPS|TERMINAL_REASON|CASH           |TOT_ASSETS     |TERMINAL_REWARD_unsc|GAINLOSS_PCT|CASH_PROPORTION
   1| 227|Last Date      |￥480,861       |￥500,760       |0.00067%  |0.15208%  |96.03%    
Eval num_timesteps=500, episode_reward=-0.00 +/- 0.00
Episode length: 228.00 +/- 0.00
New best mean reward!
   2| 357|CASH SHORTAGE  |￥2,653         |￥432,646       |-0.08472% |-13.47086%|0.61%     
   3| 314|CASH SHORTAGE  |￥6,805         |￥448,623       |-0.08554% |-10.27540%|1.52%     
   3| 227|Last Date      |￥483,659       |￥502,727       |0.00240%  |0.54544%  |96.21%    
Eval num_timesteps=1000, episode_reward=-0.00 +/- 0.00
Episode length: 228.00 +/- 0.00
New best mean reward!
   4| 316|CASH SHORTAGE  |￥11,876        |￥521,571       |-0.04485% |4.31420%  |2.28%     
   5| 227|Last Date      |￥484,481       |￥502,183       |0.00192%  |0.43654%  |96.48%    
Eval num_timesteps=1500, episode_reward=-0.00 +/- 0.00
Episode length: 228.00 +/- 0.00
   5| 342|CASH SHORTAGE  |￥5,598      

  40| 291|CASH SHORTAGE  |￥1,946         |￥501,941       |-0.06632% |0.38820%  |0.39%     
  41| 206|CASH SHORTAGE  |￥7,704         |￥450,810       |-0.12781% |-9.83798% |1.71%     
  53| 227|Last Date      |￥450,054       |￥501,073       |0.00095%  |0.21462%  |89.82%    
Eval num_timesteps=13500, episode_reward=0.00 +/- 0.00
Episode length: 228.00 +/- 0.00
New best mean reward!
  42| 287|CASH SHORTAGE  |￥3,348         |￥405,458       |-0.12006% |-18.90837%|0.83%     
  55| 227|Last Date      |￥447,708       |￥501,622       |0.00143%  |0.32431%  |89.25%    
Eval num_timesteps=14000, episode_reward=0.00 +/- 0.00
Episode length: 228.00 +/- 0.00
New best mean reward!
  43| 406|CASH SHORTAGE  |￥2,524         |￥454,601       |-0.06591% |-9.07971% |0.56%     
  44| 323|CASH SHORTAGE  |￥8,696         |￥332,534       |-0.13949% |-33.49320%|2.62%     
  57| 227|Last Date      |￥448,418       |￥499,640       |-0.00032% |-0.07207% |89.75%    
Eval num_timesteps=14500, episode_reward=0.00 +/- 0.00

KeyboardInterrupt: 

In [ ]:
model.save("different1_24.model")

### Trade

DRL model needs to update periodically in order to take full advantage of the data, ideally we need to retrain our model yearly, quarterly, or monthly. We also need to tune the parameters along the way, in this notebook I only use the in-sample data from 2009-01 to 2018-12 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [ ]:
trade.head()

In [ ]:
e_trade_gym.hmax = 5000

In [ ]:
print(len(e_trade_gym.dates))

In [ ]:
df_account_value, df_actions = DRLAgent.DRL_prediction(model=model,environment = e_trade_gym)

In [ ]:
df_actions.head()

In [ ]:
df_account_value.shape

In [ ]:
df_account_value.head(50)

<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [ ]:
print("==============Get Backtest Results===========")
perf_stats_all = backtest_stats(account_value=df_account_value, value_col_name = 'total_assets')

<a id='6.2'></a>
## 7.2 BackTestPlot

In [ ]:
print("==============Compare to DJIA===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = '^DJI', 
             baseline_start = '2016-01-01',
             baseline_end = '2021-01-01', value_col_name = 'total_assets')